用到了两个数据集，大数据集用来实现推荐系统，小数据集用来建立个性化推荐系统

In [244]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

In [114]:
md = pd. read_csv('../movies_metadata.csv')
md.head()

/Users/yangmei/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


最简单的推荐系统：越受欢迎的电影越可能被大多数人喜欢。根据popularity(和genre)推荐。排序即可。不包含个性化推荐。

In [115]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [116]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

要入排行榜的电影票数要多于95%的电影

In [119]:
m = vote_counts.quantile(0.95)
m

434.0

In [120]:
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [121]:
set(md['vote_count'].isnull()),set(md['vote_average'].isnull())

({False, True}, {False, True})

In [122]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]

In [123]:
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')

In [124]:
qualified.shape

(2274, 6)

In [125]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [126]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [127]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [19]:
qualified

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.1081,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.2135,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.8696,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.0707,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.95,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.6454,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.3244,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.3072,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.4235,"[Adventure, Fantasy, Action]",7.851924


Inception、The Dark Knight、Interstellar都是诺兰的，体现用户偏差。
根据类别构建排行榜

In [128]:
s = md['genres'].apply(lambda x : pd.Series(x)).stack().reset_index(level=1, drop=True)

In [129]:
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [130]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified

In [131]:
set(gen_md['genre'])

{nan,
 'Comedy',
 'Mardock Scramble Production Committee',
 'Family',
 'Western',
 'Action',
 'Thriller',
 'Music',
 'Documentary',
 'TV Movie',
 'Science Fiction',
 'Odyssey Media',
 'Foreign',
 'GoHands',
 'War',
 'Mystery',
 'Romance',
 'Fantasy',
 'BROSTA TV',
 'Vision View Entertainment',
 'The Cartel',
 'Adventure',
 'Rogue State',
 'Carousel Productions',
 'Sentai Filmworks',
 'Pulser Productions',
 'Crime',
 'Telescene Film Group Productions',
 'Animation',
 'Horror',
 'History',
 'Aniplex',
 'Drama'}

In [132]:
build_chart('Romance').head(15)

,title,year,vote_count,vote_average,popularity,wr
10309,Dilwale Dulhania Le Jayenge,1995,661,9,34.457,8.565285
351,Forrest Gump,1994,8147,8,48.3072,7.971357
876,Vertigo,1958,1162,8,18.2082,7.811667
40251,Your Name.,2016,1030,8,34.461252,7.789489
883,Some Like It Hot,1959,835,8,11.8451,7.745154
1132,Cinema Paradiso,1988,834,8,14.177,7.744878
19901,Paperman,2012,734,8,7.19863,7.713951
37863,Sing Street,2016,669,8,10.672862,7.689483
882,The Apartment,1960,498,8,11.9943,7.599317
38718,The Handmaiden,2016,453,8,16.727405,7.566166


基于内容的推荐
I will build two Content Based Recommenders based on:

Movie Overviews and Taglines
Movie Cast, Crew, Keywords and Genre

In [133]:
links_small = pd.read_csv('../links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
links_small

0          862
1         8844
2        15602
3        31357
4        11862
5          949
6        11860
7        45325
8         9091
9          710
10        9087
11       12110
12       21032
13       10858
14        1408
15         524
16        4584
17           5
18        9273
19       11517
20        8012
21        1710
22        9691
23       12665
24         451
25       16420
26        9263
27       17015
28         902
29       37557
         ...  
9095    308531
9096    341012
9097    259693
9098    328429
9099     43074
9100    302699
9101    324668
9102    278348
9103    258489
9104    316727
9105    316023
9106    177869
9107    353571
9108    399106
9109    328387
9110    368620
9111    401387
9112    373348
9113    338766
9114    390734
9115    314420
9116    390989
9117    159550
9118    410612
9119    392572
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [47]:
links_small.shape, md.shape, md.columns

((9112,),
 (45466, 25),
 Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
        'imdb_id', 'original_language', 'original_title', 'overview',
        'popularity', 'poster_path', 'production_companies',
        'production_countries', 'release_date', 'revenue', 'runtime',
        'spoken_languages', 'status', 'tagline', 'title', 'video',
        'vote_average', 'vote_count', 'year'],
       dtype='object'))

In [134]:
md = md.drop([19730, 29503, 35587])
md['id'] = md['id'].astype('int')#????

In [135]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

9099条数据进行个性化推荐

In [136]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [137]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [138]:
len(tf.vocabulary_)

268124

In [139]:
tf.stop_words_

set()

In [140]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [141]:
cosine_sim[0]

array([ 1.        ,  0.00680476,  0.        , ...,  0.        ,
        0.00344913,  0.        ])

In [142]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
Heat                                                     5
Sabrina                                                  6
Tom and Huck                                             7
Sudden Death                                             8
GoldenEye                                                9
The American President                                  10
Dracula: Dead and Loving It                             11
Balto                                                   12
Nixon                                                   13
Cutthroat Island                                        14
Casino                                                  15
Sense and Sensibility                             

In [143]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
#     print(len(sim_scores))
    sim_scores = [ x for x in sim_scores if isinstance(x[1], float)]
#     print(len(sim_scores))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [144]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

In [89]:
get_recommendations('The Godfather').head(10)

9099
9099


973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

 recommender that takes genre, keywords, cast and crew into consideration.

In [145]:
credits = pd.read_csv('../credits.csv')
keywords = pd.read_csv('../keywords.csv')

In [146]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [104]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
cast    45476 non-null object
crew    45476 non-null object
id      45476 non-null int64
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [105]:
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [108]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
id          46419 non-null int64
keywords    46419 non-null object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [147]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,status,tagline,title,video,vote_average,vote_count,year,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Released,NaN,Toy Story,False,7.7,5415.0,1995,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,1995,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,1995,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [155]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [158]:
def get_director(x):
    d = []
    for i in x:
        if i['job'] == 'Director':
            d.append(i['name'])
    if len(d) != 0:
        return d
    return []

In [159]:
smd['director'] = smd['crew'].apply(get_director)

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [174]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [176]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['keywords']

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
5        [robbery, detective, bank, obsession, chase, s...
6        [paris, brother brother relationship, chauffeu...
7                                                       []
8          [terrorist, hostage, explosive, vice president]
9        [cuba, falsely accused, secret identity, compu...
10       [white house, usa president, new love, widower...
11                                        [dracula, spoof]
12       [wolf, dog-sledding race, alaska, dog, goose, ...
13       [usa president, presidential election, waterga...
14       [exotic island, treasure, map, ship, scalp, pi...
15       [poker, drug abuse, 1970s, overdose, illegal p...
16       [bowling, based on novel, servant, country lif.

In [178]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [180]:
s = smd['keywords'].apply(lambda x : pd.Series(x)).stack().reset_index(level=1, drop=True)
s

0                            jealousy
0                                 toy
0                                 boy
0                          friendship
0                             friends
0                             rivalry
0                       boy next door
0                             new toy
0                   toy comes to life
1                          board game
1                       disappearance
1            based on children's book
1                            new home
1                             recluse
1                        giant insect
2                             fishing
2                         best friend
2                duringcreditsstinger
2                             old men
3                      based on novel
3            interracial relationship
3                       single mother
3                             divorce
3                         chick flick
4                                baby
4                      midlife crisis
4           

In [181]:
s.name = 'keywords'
s = s.value_counts()

In [183]:
s = s[s > 1]
s

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
violence                264
love                    222
musical                 219
sex                     219
suspense                212
revenge                 208
friendship              202
nudity                  197
police                  195
sport                   192
dystopia                184
biography               178
aftercreditsstinger     178
sequel                  160
prison                  155
new york                150
female nudity           147
drug                    144
teenager                141
los angeles             134
gay                     132
high school             125
world war ii            121
paris                   118
family                  116
                       ... 
hook for a hand           2
bar shootout              2
episode film              2
somalia                   2
talking             

In [184]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [201]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [202]:
smd['keywords'] = smd['keywords'].apply(filter_keywords) #过滤只出现一次的keywords
smd['keywords']

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2             [fishing, best friend, duringcreditsstinger]
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
5        [robbery, detective, bank, obsession, chase, s...
6        [paris, brother brother relationship, chauffeu...
7                                                       []
8          [terrorist, hostage, explosive, vice president]
9        [cuba, falsely accused, secret identity, compu...
10         [white house, usa president, new love, widower]
11                                        [dracula, spoof]
12                 [wolf, alaska, dog, goose, bear attack]
13       [usa president, presidential election, waterga...
14       [exotic island, treasure, map, ship, scalp, pi...
15       [poker, drug abuse, 1970s, overdose, illegal p...
16       [bowling, based on novel, servant, country lif.

In [203]:
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords']

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [jealousi, toy, boy, friendship, friend, rival...
1        [board gam, disappear, based on children's boo...
2                  [fish, best friend, duringcreditssting]
3        [based on novel, interracial relationship, sin...
4        [babi, midlife crisi, confid, age, daughter, m...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brother brother relationship, chauffeur...
7                                                       []
8                 [terrorist, hostag, explos, vice presid]
9        [cuba, falsely accus, secret ident, computer v...
10                [white hous, usa presid, new lov, widow]
11                                        [dracula, spoof]
12                  [wolf, alaska, dog, goos, bear attack]
13       [usa presid, presidential elect, watergate sca...
14       [exotic island, treasur, map, ship, scalp, pirat]
15       [poker, drug abus, 1970s, overdos, illegal pro...
16       [bowl, based on novel, servant, country lif, j.

In [204]:
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['keywords']

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2                   [fish, bestfriend, duringcreditssting]
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brotherbrotherrelationship, chauffeur, ...
7                                                       []
8                  [terrorist, hostag, explos, vicepresid]
9        [cuba, falselyaccus, secretident, computerviru...
10                   [whitehous, usapresid, newlov, widow]
11                                        [dracula, spoof]
12                   [wolf, alaska, dog, goos, bearattack]
13       [usapresid, presidentialelect, watergatescand,...
14        [exoticisland, treasur, map, ship, scalp, pirat]
15       [poker, drugabus, 1970s, overdos, illegalprost...
16       [bowl, basedonnovel, servant, countrylif, jane.

In [206]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] 

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [jealousi, toy, boy, friendship, friend, rival...
1        [boardgam, disappear, basedonchildren'sbook, n...
2        [fish, bestfriend, duringcreditssting, walterm...
3        [basedonnovel, interracialrelationship, single...
4        [babi, midlifecrisi, confid, age, daughter, mo...
5        [robberi, detect, bank, obsess, chase, shoot, ...
6        [pari, brotherbrotherrelationship, chauffeur, ...
7        [jonathantaylorthomas, bradrenfro, rachaelleig...
8        [terrorist, hostag, explos, vicepresid, jean-c...
9        [cuba, falselyaccus, secretident, computerviru...
10       [whitehous, usapresid, newlov, widow, michaeld...
11       [dracula, spoof, leslienielsen, melbrooks, amy...
12       [wolf, alaska, dog, goos, bearattack, kevinbac...
13       [usapresid, presidentialelect, watergatescand,...
14       [exoticisland, treasur, map, ship, scalp, pira...
15       [poker, drugabus, 1970s, overdos, illegalprost...
16       [bowl, basedonnovel, servant, countrylif, jane.

In [207]:
smd['soup'][0]

['jealousi',
 'toy',
 'boy',
 'friendship',
 'friend',
 'rivalri',
 'boynextdoor',
 'newtoy',
 'toycomestolif',
 'tomhanks',
 'timallen',
 'donrickles',
 'johnlasseter',
 'Animation',
 'Comedy',
 'Family']

In [216]:
smd.iloc[0][['keywords', 'cast', 'director', 'genres']]

keywords    [jealousi, toy, boy, friendship, friend, rival...
cast                         [tomhanks, timallen, donrickles]
director                                       [johnlasseter]
genres                            [Animation, Comedy, Family]
Name: 0, dtype: object

In [217]:
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [219]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [221]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [223]:
smd = smd.reset_index()

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,vote_average,vote_count,year,cast,crew,keywords,cast_size,crew_size,director,soup
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,7.7,5415.0,1995,"[tomhanks, timallen, donrickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousi, toy, boy, friendship, friend, rival...",13,106,[johnlasseter],jealousi toy boy friendship friend rivalri boy...
1,1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,...,6.9,2413.0,1995,"[robinwilliams, jonathanhyde, kirstendunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[boardgam, disappear, basedonchildren'sbook, n...",26,16,[joejohnston],boardgam disappear basedonchildren'sbook newho...
2,2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,...,6.5,92.0,1995,"[waltermatthau, jacklemmon, ann-margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fish, bestfriend, duringcreditssting]",7,4,[howarddeutch],fish bestfriend duringcreditssting waltermatth...
3,3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,...,6.1,34.0,1995,"[whitneyhouston, angelabassett, lorettadevine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[basedonnovel, interracialrelationship, single...",10,10,[forestwhitaker],basedonnovel interracialrelationship singlemot...
4,4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,...,5.7,173.0,1995,"[stevemartin, dianekeaton, martinshort]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[babi, midlifecrisi, confid, age, daughter, mo...",12,7,[charlesshyer],babi midlifecrisi confid age daughter motherda...
5,5,False,NaN,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,...,7.7,1886.0,1995,"[alpacino, robertdeniro, valkilmer]","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[robberi, detect, bank, obsess, chase, shoot, ...",65,71,[michaelmann],robberi detect bank obsess chase shoot thief h...
6,6,False,NaN,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,...,6.2,141.0,1995,"[harrisonford, juliaormond, gregkinnear]","[{'credit_id': '52fe44959251416c75039da9', 'de...","[pari, brotherbrotherrelationship, chauffeur, ...",57,53,[sydneypollack],pari brotherbrotherrelationship chauffeur long...
7,7,False,NaN,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,...,5.4,45.0,1995,"[jonathantaylorthomas, bradrenfro, rachaelleig...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[],7,4,[peterhewitt],jonathantaylorthomas bradrenfro rachaelleighco...
8,8,False,NaN,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,...,5.5,174.0,1995,"[jean-claudevandamme, powersboothe, dorianhare...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[terrorist, hostag, explos, vicepresid]",6,9,[peterhyams],terrorist hostag explos vicepresid jean-claude...
9,9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[Adventure, Action, Thriller]",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,...,6.6,1194.0,1995,"[piercebrosnan, seanbean, izabellascorupco]","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...","[cuba, falselyaccus, secretident, computerviru...",20,46,[martincampbell],cuba falselyaccus secretident computervirus se...


In [224]:
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [231]:
cosine_sim[0]

array([ 1.        ,  0.03456506,  0.04120428, ...,  0.        ,
        0.        ,  0.        ])

In [228]:
get_recommendations('The Dark Knight').head(10)

8031         The Dark Knight Rises
6218                 Batman Begins
7659    Batman: Under the Red Hood
8927       Kidnapping Mr. Heineken
1134                Batman Returns
5943                      Thursday
1260                Batman & Robin
2448                    Nighthawks
8026            Bullet to the Head
9121                  İtirazım Var
Name: title, dtype: object

In [232]:
get_recommendations('Mean Girls').head(10)

8883                                   The DUFF
7377                    I Love You, Beth Cooper
4763                              Freaky Friday
6698                      It's a Boy Girl Thing
3712                       The Princess Diaries
7494    American Pie Presents: The Book of Love
5542                                  Sleepover
5163                       Just One of the Guys
3319                            Head Over Heels
5092                           Lord Love a Duck
Name: title, dtype: object

In [237]:
def improved_recommendations(title):
    title = 'The Dark Knight'
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [238]:
improved_recommendations('The Dark Knight')


/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/yangmei/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,title,vote_count,vote_average,year,wr
8031,The Dark Knight Rises,9263,7,2012,6.921448
6218,Batman Begins,7511,7,2005,6.904127
7659,Batman: Under the Red Hood,459,7,2010,6.147016
1134,Batman Returns,1706,6,1992,5.846862
7561,Harry Brown,351,6,2009,5.582529
6645,Harsh Times,198,6,2005,5.481464
7582,Defendor,197,6,2009,5.480642
8026,Bullet to the Head,490,5,2013,5.115027
9024,Batman v Superman: Dawn of Justice,7189,5,2016,5.013943
1260,Batman & Robin,1447,4,1997,4.287233


In [239]:
ratings = pd.read_csv('../ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [245]:
reader = Reader()
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [247]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9025
MAE:  0.6942
------------
Fold 2
RMSE: 0.8936
MAE:  0.6901
------------
Fold 3
RMSE: 0.9000
MAE:  0.6918
------------
Fold 4
RMSE: 0.8954
MAE:  0.6879
------------
Fold 5
RMSE: 0.8902
MAE:  0.6853
------------
------------
Mean RMSE: 0.8964
Mean MAE : 0.6899
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.69423146387757706,
                             0.69010616711499062,
                             0.6917521711145912,
                             0.68794981391582233,
                             0.68530902722480103],
                            'rmse': [0.902539226847068,
                             0.89358370169211843,
                             0.90004966758731619,
                             0.89542441446804444,
                             0.8901649063838929]})

In [248]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [249]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [250]:
svd.predict(1, 302, 3)


Prediction(uid=1, iid=302, r_ui=3, est=2.8562289278927793, details={'was_impossible': False})

In [252]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [253]:
id_map = pd.read_csv('../links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')

In [255]:
indices_map = id_map.set_index('id')
indices_map

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5
949.0,6
11860.0,7
45325.0,8
9091.0,9


In [257]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [258]:
hybrid(1, 'Avatar')


,title,vote_count,vote_average,year,id,est
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,2.982123
7705,Alice in Wonderland,8.0,5.4,1933,25694,2.915238
2014,Fantastic Planet,140.0,7.6,1973,16306,2.810453
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,2.808146
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.761818
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.759896
7091,Mutant Chronicles,142.0,5.1,2008,13256,2.685536
4017,Hawk the Slayer,13.0,4.5,1980,25628,2.671984
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.647908
831,Escape to Witch Mountain,60.0,6.5,1975,14821,2.645605


In [259]:
hybrid(500, 'Avatar')


,title,vote_count,vote_average,year,id,est
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.155675
7705,Alice in Wonderland,8.0,5.4,1933,25694,3.063222
7088,Star Wars: The Clone Wars,434.0,5.8,2008,12180,2.982932
6084,Beastmaster 2: Through the Portal of Time,17.0,4.6,1991,27549,2.961817
831,Escape to Witch Mountain,60.0,6.5,1975,14821,2.931613
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,2.911105
3060,Sinbad and the Eye of the Tiger,39.0,6.3,1977,11940,2.894881
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,2.875041
2014,Fantastic Planet,140.0,7.6,1973,16306,2.825676
1668,Return from Witch Mountain,38.0,5.6,1978,14822,2.808026
